In [4]:
import xarray as xr

In [23]:
cap = xr.open_dataset('../build/capacity_factors/capacity_factors_offshore_deep_awe.nc')

In [24]:
cap

<xarray.Dataset>
Dimensions:                        (time: 52584, id: 31)
Coordinates:
  * time                           (time) datetime64[ns] 2013-01-01 ... 2018-...
  * id                             (id) object '3' '6' '15' ... '276' '280'
Data variables:
    __xarray_dataarray_variable__  (time, id) float64 ...

In [18]:
av = xr.open_dataset("../build/availability/availability_offshore_deep.nc")

In [22]:
av.stack(spatial=["y", "x"])

<xarray.Dataset>
Dimensions:                        (id: 31, spatial: 34937)
Coordinates:
  * id                             (id) object '3' '6' '15' ... '276' '280'
  * spatial                        (spatial) object MultiIndex
  * y                              (spatial) float64 72.0 72.0 ... 32.0 32.0
  * x                              (spatial) float64 -17.0 -16.75 ... 36.75 37.0
Data variables:
    __xarray_dataarray_variable__  (id, spatial) float64 0.0 0.0 0.0 ... 0.0 0.0